## キーワードで検索

In [1]:
word1 = "生活保護"
word2 = "尾道市"
word3 = ""

In [2]:
import os
import re

# 空白（全角・半角）を削除したバージョンも用意
def remove_spaces(text):
    return re.sub(r'[\s\u3000]', '', text)  # \s: 半角空白、\u3000: 全角空白

# 検索対象ディレクトリ
data_dir = "/work/a06/rmori/spillover/data"

# 条件に一致するファイルパスを格納
matching_paths = []

# 再帰的にCSVファイルを探索
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith(".csv"):
            full_path = os.path.join(root, file)
            try:
                with open(full_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                    stripped = remove_spaces(content)
                    if (word1 in stripped) and (word2 in stripped) and (word3 in stripped):
                        matching_paths.append(full_path)
            except UnicodeDecodeError:
                try:
                    with open(full_path, 'r', encoding='shift_jis') as f:
                        content = f.read()
                        stripped = remove_spaces(content)
                        if (word1 in stripped) and (word2 in stripped) and (word3 in stripped):
                            matching_paths.append(full_path)
                except Exception as e:
                    print(f"❌ 読込失敗: {full_path} ({e})")

# 結果を表示
for path in matching_paths:
    print(path)



/work/a06/rmori/spillover/data/1982/tone-s18.csv
/work/a06/rmori/spillover/data/1982/tone-c01.csv
/work/a06/rmori/spillover/data/1988/tone-c01.csv
/work/a06/rmori/spillover/data/1988/tone-s18.csv
/work/a06/rmori/spillover/data/1966/tone-v25.csv
/work/a06/rmori/spillover/data/1966/tone-a01.csv
/work/a06/rmori/spillover/data/1975/tone-r30.csv
/work/a06/rmori/spillover/data/1975/tone-a01.csv
/work/a06/rmori/spillover/data/1972/18回(昭和47年版） .csv
/work/a06/rmori/spillover/data/1972/tone-a01.csv
/work/a06/rmori/spillover/data/1972/tone-r27.csv
/work/a06/rmori/spillover/data/1978/tone-c01.csv
/work/a06/rmori/spillover/data/1978/tone-r30.csv
/work/a06/rmori/spillover/data/1959/tone-a01.csv
/work/a06/rmori/spillover/data/1959/tone-v22.csv
/work/a06/rmori/spillover/data/1985/tone-s18.csv
/work/a06/rmori/spillover/data/1985/tone-c01.csv
/work/a06/rmori/spillover/data/1961/tone-a01.csv
/work/a06/rmori/spillover/data/1961/tone-v23.csv
/work/a06/rmori/spillover/data/1965/tone-v23.csv
/work/a06/rmori/

In [3]:
matching_paths = sorted(matching_paths)


In [5]:
year = "1959"
path_year = []
for path in matching_paths:
    if year in path:
        path_year.append(path)
path_year = sorted(path_year)
print(path_year)

['/work/a06/rmori/spillover/data/1959/tone-a01.csv', '/work/a06/rmori/spillover/data/1959/tone-v22.csv']


In [26]:
import os
import pandas as pd
import re

results = []

for path in sorted(matching_paths):
    match = re.search(r"(\d{4})", path)
    if not match:
        continue
    year = int(match.group(1))

    try:
        df = pd.read_csv(path, header=None, encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(path, header=None, encoding='shift_jis', errors='ignore')
    except Exception as e:
        print(f"❌ 読込失敗: {path} ({e})")
        continue

    row_index = None
    col_index = None

    # 広島市の行を探す（上から）
    for r in range(df.shape[0]):
        for c in range(df.shape[1]):
            val = df.iat[r, c]
            if pd.isna(val):
                continue
            cleaned = str(val).replace(' ', '').replace('　', '')
            if "広島市" in cleaned:
                row_index = r
                break
        if row_index is not None:
            break

    if row_index is None:
        print(f"⚠️ 広島市が見つからない: {path}")
        continue

    # 広島市より上の行から、耕地面積がある「最後の列番号」を探す
    for r in range(row_index):  # row_index未満の行のみ対象
        for c in range(df.shape[1]):
            val = df.iat[r, c]
            if pd.isna(val):
                continue
            cleaned = str(val).replace(' ', '').replace('　', '')
            if "耕地面積" in cleaned:
                col_index = c  # 更新し続けて「最後の列番号」にする

    if col_index is None:
        print(f"⚠️ 製造業が見つからない: {path}")
        continue

    # 広島市の行・その列の交差セルを取得
    value = df.iat[row_index, col_index]
    results.append((year, value))

# 年でソート
results.sort(key=lambda x: x[0])

# 結果出力
for year, value in results:
    print(f"{year}: {value}")


⚠️ 製造業が見つからない: /work/a06/rmori/spillover/data/1957/tone-b11.csv
⚠️ 製造業が見つからない: /work/a06/rmori/spillover/data/1959/tone-c09.csv
⚠️ 製造業が見つからない: /work/a06/rmori/spillover/data/1971/17回(昭和46年版） .csv
⚠️ 製造業が見つからない: /work/a06/rmori/spillover/data/1972/18回(昭和47年版） .csv
1954: 607
1954: 広島市
1954: 607.15
1956: nan
1956: 6663
1957: nan
1957: nan
1958: 49862.91
1958: 　　なお広島市宇品島，安芸郡江田島町一ツ子島,二ツ子島・御調郡向島町岡島・沼隈郡沼隈町茂美島を除く．
1958: 729.63
1958: 3207
1958: 広島市
1958: 広島市
1959: nan
1959: 723.6
1959: 3207
1959: 7393
1959: 1003
1960: nan
1960: ４　　航路回数は旅客航路免許回数である。なお広島市宇品島・安芸郡江田島町一ツ子島,二ツ子島・御調郡向島町岡島・沼隈郡沼隈町茂美島を除く。
1960: 0.23
1960: 1
1960: 広島市
1960: 1
1960: 0.23
1961: nan
1961: ４　　航路回数は旅客航路免許回数である。なお広島市宇品島・安芸郡江田島町一ツ子島,二ツ子島・御調郡向島町岡島・沼隈郡沼隈町茂美島を除く。
1961: 0.23
1961: 1
1961: 広島市
1961: 1
1961: 0.23
1962: 0.23
1962: ４　　航路回数は旅客航路免許回数である．なお広島市宇品島・安芸郡江田島町
      一ツ子島,二ツ子島・御調郡向島町岡島・沼隈郡沼隈町茂美島を除く．
1962: 1
1962: 1
1962: 0.23
1963: 0.23
1963: 4 　航路回数は旅客航路免許回数である．なお広島市宇品島・安芸郡江田島町一ツ子島，二ツ子島・御調郡向島町岡島・沼隈郡沼隈町茂美島を除く．
1963: 0.23
1963: 0.

In [15]:
results(year>=1970)

TypeError: 'list' object is not callable

In [21]:
results

[(1958, '32690'),
 (1958, '15408'),
 (1958, '15408'),
 (1958, '広島市'),
 (1959, '15408'),
 (1959, '15408'),
 (1959, nan),
 (1960, nan),
 (1960, '3.4'),
 (1961, nan),
 (1961, '3.4'),
 (1962, '14596'),
 (1962, '3.4'),
 (1963, '14596'),
 (1963, '3.4'),
 (1964, nan),
 (1964, '3.4'),
 (1965, '10252'),
 (1965, nan),
 (1966, '10252'),
 (1966, '2'),
 (1967, '10252'),
 (1967, '2'),
 (1968, '10252'),
 (1968, '広島市'),
 (1968, '広島市'),
 (1968, '2'),
 (1969, '10252'),
 (1969, '2'),
 (1970, '7849'),
 (1970, '7849'),
 (1971, '19950'),
 (1971, '19950'),
 (1972, '50656'),
 (1972, '19950'),
 (1973, '57572'),
 (1973, '19950'),
 (1974, '62760'),
 (1974, '19950'),
 (1975, '52418'),
 (1975, nan),
 (1976, '52418'),
 (1976, nan),
 (1977, '52418'),
 (1977, '1'),
 (1978, '52418'),
 (1978, '1'),
 (1979, '52418'),
 (1979, nan),
 (1980, '48385'),
 (1980, nan),
 (1981, '48385'),
 (1981, '1'),
 (1982, '48385'),
 (1982, '1'),
 (1983, '48385'),
 (1983, '1'),
 (1984, '48385'),
 (1984, '1'),
 (1985, '49345'),
 (1985, '1'),


In [28]:
filtered_results = [(year, value) for year, value in results if year>=1975]
filtered_results

[(1975, '4089'),
 (1975, '6291'),
 (1975, '51147'),
 (1976, '4089'),
 (1976, '6291'),
 (1976, '51147'),
 (1977, '4089'),
 (1977, '51147'),
 (1978, '4089'),
 (1978, '3186'),
 (1978, '51147'),
 (1979, '4089'),
 (1979, '6291'),
 (1979, '1'),
 (1979, '51147'),
 (1980, '3703'),
 (1980, '6543'),
 (1980, '1'),
 (1980, '48385'),
 (1981, '66801'),
 (1981, '6543'),
 (1981, '48385'),
 (1982, '11678'),
 (1982, '6543'),
 (1982, '48385'),
 (1983, '480078'),
 (1983, '6543'),
 (1983, '48385'),
 (1984, '3703'),
 (1984, '6543'),
 (1984, '48385'),
 (1985, '29.7'),
 (1985, '7130'),
 (1985, '49345'),
 (1986, '93.9'),
 (1986, '7130'),
 (1986, '49345'),
 (1987, '3645'),
 (1987, '7130'),
 (1987, '49345'),
 (1988, '3645'),
 (1988, '7130'),
 (1989, '3645'),
 (1989, '7130')]

## 同じファイルで検索

In [136]:
import os

# 検索対象ディレクトリ
data_dir = "/work/a06/rmori/spillover/data"

# 条件に一致するファイルパスを格納
matching_paths2 = []

# 再帰的にCSVファイルを探索
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if "tone-c01" in file:
            full_path = os.path.join(root, file)
            matching_paths2.append(full_path)

# 結果を表示
for path in matching_paths2:
    print(path)


/work/a06/rmori/spillover/data/1982/tone-c01.csv
/work/a06/rmori/spillover/data/1988/tone-c01.csv
/work/a06/rmori/spillover/data/1966/tone-c01.csv
/work/a06/rmori/spillover/data/1954/tone-c01.csv
/work/a06/rmori/spillover/data/1975/tone-c01.csv
/work/a06/rmori/spillover/data/1972/tone-c01.csv
/work/a06/rmori/spillover/data/1978/tone-c01.csv
/work/a06/rmori/spillover/data/1959/tone-c01.csv
/work/a06/rmori/spillover/data/1985/tone-c01.csv
/work/a06/rmori/spillover/data/1961/tone-c01.csv
/work/a06/rmori/spillover/data/1965/tone-c01.csv
/work/a06/rmori/spillover/data/1981/tone-c01.csv
/work/a06/rmori/spillover/data/1957/tone-c01.csv
/work/a06/rmori/spillover/data/1976/tone-c01.csv
/work/a06/rmori/spillover/data/1962/tone-c01.csv
/work/a06/rmori/spillover/data/1968/tone-c01.csv
/work/a06/rmori/spillover/data/1986/tone-c01.csv
/work/a06/rmori/spillover/data/1958/tone-c01.csv
/work/a06/rmori/spillover/data/1960/tone-c01.csv
/work/a06/rmori/spillover/data/1984/tone-c01.csv
/work/a06/rmori/spil

In [138]:
matching_paths2 = sorted(matching_paths2)
matching_paths2 = matching_paths2[15:]
matching_paths2

['/work/a06/rmori/spillover/data/1970/tone-c01.csv',
 '/work/a06/rmori/spillover/data/1972/tone-c01.csv',
 '/work/a06/rmori/spillover/data/1973/tone-c01.csv',
 '/work/a06/rmori/spillover/data/1974/tone-c01.csv',
 '/work/a06/rmori/spillover/data/1975/tone-c01.csv',
 '/work/a06/rmori/spillover/data/1976/tone-c01.csv',
 '/work/a06/rmori/spillover/data/1977/tone-c01.csv',
 '/work/a06/rmori/spillover/data/1978/tone-c01.csv',
 '/work/a06/rmori/spillover/data/1979/tone-c01.csv',
 '/work/a06/rmori/spillover/data/1980/tone-c01.csv',
 '/work/a06/rmori/spillover/data/1981/tone-c01.csv',
 '/work/a06/rmori/spillover/data/1982/tone-c01.csv',
 '/work/a06/rmori/spillover/data/1983/tone-c01.csv',
 '/work/a06/rmori/spillover/data/1984/tone-c01.csv',
 '/work/a06/rmori/spillover/data/1985/tone-c01.csv',
 '/work/a06/rmori/spillover/data/1986/tone-c01.csv',
 '/work/a06/rmori/spillover/data/1987/tone-c01.csv',
 '/work/a06/rmori/spillover/data/1988/tone-c01.csv',
 '/work/a06/rmori/spillover/data/1989/tone-c01

In [143]:
import os
import pandas as pd
import re

results = []

for path in sorted(matching_paths2):
    match = re.search(r"(\d{4})", path)
    if not match:
        continue
    year = int(match.group(1))

    try:
        df = pd.read_csv(path, header=None, encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(path, header=None, encoding='shift_jis', errors='ignore')
    except Exception as e:
        print(f"❌ 読込失敗: {path} ({e})")
        continue

    row_index = None
    col_index = None

    # 広島市の行を探す（上から）
    for r in range(df.shape[0]):
        for c in range(df.shape[1]):
            val = df.iat[r, c]
            if pd.isna(val):
                continue
            cleaned = str(val).replace(' ', '').replace('　', '')
            if "広島市" in cleaned:
                row_index = r
                break
        if row_index is not None:
            break

    if row_index is None:
        print(f"⚠️ 広島市が見つからない: {path}")
        continue

    # 広島市より上の行から、耕地面積がある「最後の列番号」を探す
    for r in range(row_index):  # row_index未満の行のみ対象
        for c in range(df.shape[1]):
            val = df.iat[r, c]
            if pd.isna(val):
                continue
            cleaned = str(val).replace(' ', '').replace('　', '')
            if "就業者数" in cleaned:
                col_index = c  # 更新し続けて「最後の列番号」にする

    if col_index is None:
        print(f"⚠️ 製造業が見つからない: {path}")
        continue

    # 広島市の行・その列の交差セルを取得
    value = df.iat[row_index, col_index+3]
    #value2 = df.iat[row_index, col_index]
    results.append((year, value))

# 年でソート
results.sort(key=lambda x: x[0])

# 結果出力
for year, value in results:
    print(f"{year}: {value}")

⚠️ 製造業が見つからない: /work/a06/rmori/spillover/data/1970/tone-c01.csv
⚠️ 製造業が見つからない: /work/a06/rmori/spillover/data/1972/tone-c01.csv
⚠️ 製造業が見つからない: /work/a06/rmori/spillover/data/1973/tone-c01.csv
⚠️ 製造業が見つからない: /work/a06/rmori/spillover/data/1974/tone-c01.csv
⚠️ 製造業が見つからない: /work/a06/rmori/spillover/data/1975/tone-c01.csv
⚠️ 製造業が見つからない: /work/a06/rmori/spillover/data/1976/tone-c01.csv
1977: 62.8
1978: 62.8
1979: 62.8
1980: 62.8
1981: 67.4
1982: 67.4
1983: 67.4
1984: 67.4
1985: 67.3
1986: 68.7
1987: 68.7
1988: 68.7
1989: 68.7


In [146]:
df2 = pd.read_csv('/work/a06/rmori/spillover/data/hiroshima.csv', index_col=0)
df2

,year,製造業事業所数,製造業従業者数,製造品出荷額,商店数,従業者数,商品販売額,飲食店商店数,飲食店常時従業員数,飲食店商品販売額,第1次産業,第2次産業,第3次産業
0,1977,2828,73569,101863627,15741.0,103586.0,429912581.0,6136.0,24741.0,7923231.0,3.3,33.5,62.8
1,1978,2761,70280,119175646,15741.0,103586.0,429912581.0,6136.0,24741.0,7923231.0,3.3,33.5,62.8
2,1979,2827,68762,124779306,17193.0,111992.0,561629968.0,5135.0,17410.0,6934025.0,3.3,33.5,62.8
3,1980,2797,66591,145556386,17194.0,112529.0,5648084.0,5141.0,17416.0,68641.0,3.3,33.5,62.8
4,1981,2794,69327,158359189,17194.0,112529.0,5648084.0,5141.0,17416.0,68641.0,2.8,29.6,67.4
5,1982,2091,67294,181238210,18310.0,118561.0,7767100.0,5275.0,18382.0,84425.0,2.8,29.6,67.4
6,1983,2275,67520,170571515,18307.0,118547.0,7671348.0,5275.0,18382.0,83688.0,2.8,29.6,67.4
7,1984,2368,67472,163108215,18307.0,118547.0,7671348.0,5275.0,18382.0,83688.0,2.8,29.6,67.4
8,1985,2398,69966,175231152,18160.0,119067.0,7969510.0,5595.0,19304.0,87412.0,2.9,29.7,67.3
9,1986,2414,70873,184362815,18160.0,119067.0,7969510.0,5555.0,19664.0,92734.0,2.6,28.4,68.7


In [147]:
df2.T

,0,1,2,3,4,5,6,7,8,9,10,11,12
year,1977.0,1978.0,1979.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0
製造業事業所数,2828.0,2761.0,2827.0,2797.0,2794.0,2091.0,2275.0,2368.0,2398.0,2414.0,2469.0,2379.0,2364.0
製造業従業者数,73569.0,70280.0,68762.0,66591.0,69327.0,67294.0,67520.0,67472.0,69966.0,70873.0,70558.0,68226.0,67505.0
製造品出荷額,101863627.0,119175646.0,124779306.0,145556386.0,158359189.0,181238210.0,170571515.0,163108215.0,175231152.0,184362815.0,195012502.0,200812468.0,234256959.0
商店数,15741.0,15741.0,17193.0,17194.0,17194.0,18310.0,18307.0,18307.0,18160.0,18160.0,18160.0,18632.0,18632.0
従業者数,103586.0,103586.0,111992.0,112529.0,112529.0,118561.0,118547.0,118547.0,119067.0,119067.0,119067.0,130260.0,130260.0
商品販売額,429912581.0,429912581.0,561629968.0,5648084.0,5648084.0,7767100.0,7671348.0,7671348.0,7969510.0,7969510.0,7969510.0,8723231.0,8723231.0
飲食店商店数,6136.0,6136.0,5135.0,5141.0,5141.0,5275.0,5275.0,5275.0,5595.0,5555.0,5555.0,5555.0,5316.0
飲食店常時従業員数,24741.0,24741.0,17410.0,17416.0,17416.0,18382.0,18382.0,18382.0,19304.0,19664.0,19664.0,19664.0,21362.0
飲食店商品販売額,7923231.0,7923231.0,6934025.0,68641.0,68641.0,84425.0,83688.0,83688.0,87412.0,92734.0,92734.0,92734.0,107605.0


In [144]:
#データフレームに格納
df_now = pd.DataFrame(results, columns=["year", "第3次産業"])
df2 = pd.merge(df2, df_now, on="year", how="outer")
df2

,year,製造業事業所数,製造業従業者数,製造品出荷額,商店数,従業者数,商品販売額,飲食店商店数,飲食店常時従業員数,飲食店商品販売額,第1次産業,第2次産業,第3次産業
0,1977,2828,73569,101863627,15741.0,103586.0,429912581.0,6136.0,24741.0,7923231.0,3.3,33.5,62.8
1,1978,2761,70280,119175646,15741.0,103586.0,429912581.0,6136.0,24741.0,7923231.0,3.3,33.5,62.8
2,1979,2827,68762,124779306,17193.0,111992.0,561629968.0,5135.0,17410.0,6934025.0,3.3,33.5,62.8
3,1980,2797,66591,145556386,17194.0,112529.0,5648084.0,5141.0,17416.0,68641.0,3.3,33.5,62.8
4,1981,2794,69327,158359189,17194.0,112529.0,5648084.0,5141.0,17416.0,68641.0,2.8,29.6,67.4
5,1982,2091,67294,181238210,18310.0,118561.0,7767100.0,5275.0,18382.0,84425.0,2.8,29.6,67.4
6,1983,2275,67520,170571515,18307.0,118547.0,7671348.0,5275.0,18382.0,83688.0,2.8,29.6,67.4
7,1984,2368,67472,163108215,18307.0,118547.0,7671348.0,5275.0,18382.0,83688.0,2.8,29.6,67.4
8,1985,2398,69966,175231152,18160.0,119067.0,7969510.0,5595.0,19304.0,87412.0,2.9,29.7,67.3
9,1986,2414,70873,184362815,18160.0,119067.0,7969510.0,5555.0,19664.0,92734.0,2.6,28.4,68.7


In [117]:
df1.iat[3,9] = 68641
df1

,year,製造業事業所数,製造業従業者数,製造品出荷額,商店数,従業者数,商品販売額,飲食店商店数,飲食店常時従業員数,飲食店商品販売額
0,1977,2828,73569,101863627,15741.0,103586.0,429912581.0,6136.0,24741.0,7923231.0
1,1978,2761,70280,119175646,15741.0,103586.0,429912581.0,6136.0,24741.0,7923231.0
2,1979,2827,68762,124779306,17193.0,111992.0,561629968.0,5135.0,17410.0,6934025.0
3,1980,2797,66591,145556386,17194.0,112529.0,5648084.0,5141.0,17416.0,68641.0
4,1981,2794,69327,158359189,17194.0,112529.0,5648084.0,5141.0,17416.0,68641.0
5,1982,2091,67294,181238210,18310.0,118561.0,7767100.0,5275.0,18382.0,84425.0
6,1983,2275,67520,170571515,18307.0,118547.0,7671348.0,5275.0,18382.0,83688.0
7,1984,2368,67472,163108215,18307.0,118547.0,7671348.0,5275.0,18382.0,83688.0
8,1985,2398,69966,175231152,18160.0,119067.0,7969510.0,5595.0,19304.0,87412.0
9,1986,2414,70873,184362815,18160.0,119067.0,7969510.0,5555.0,19664.0,92734.0


In [145]:
df2.to_csv('/work/a06/rmori/spillover/data/hiroshima.csv')

In [95]:
df1 = df1.rename(columns={'従業者数_x':'従業者数'})

In [101]:
df1 = df1.drop('従業者数_y', axis =1)
df1

,year,製造業事業所数,製造業従業者数,製造品出荷額,商店数,従業者数,商品販売額
0,1977,2828,73569,101863627,15741.0,103586.0,429912581.0
1,1978,2761,70280,119175646,15741.0,NaN,373458968.0
2,1979,2827,68762,124779306,17193.0,NaN,489384112.0
3,1980,2797,66591,145556386,17194.0,112529.0,5648084.0
4,1981,2794,69327,158359189,17194.0,112529.0,5648084.0
5,1982,2091,67294,181238210,18310.0,118561.0,7767100.0
6,1983,2275,67520,170571515,18307.0,118547.0,7671348.0
7,1984,2368,67472,163108215,18307.0,118547.0,7671348.0
8,1985,2398,69966,175231152,18160.0,119067.0,7969510.0
9,1986,2414,70873,184362815,18160.0,119067.0,7969510.0


KeyError: (0, 6)

# 農地面積

# 農家人口
/work/a06/rmori/spillover/data/1958/tone-g07.csv
